# Large Test Data Generator

### # Author: Vivek Maswadkar

In [ ]:
import random
import json
import string
import phonenumbers
import time

In [ ]:
with open('customer_master_parameters.json') as data_file:
    data = json.load(data_file)
separator=(data["separator"])

In [ ]:
def get_phonenumber(region_code):
    example_number = phonenumbers.example_number(region_code)
    phone_number_length = len(str(example_number.national_number))
    random_phone_number = str(random.randrange(10**int(phone_number_length-1),10**int(phone_number_length)))
    return("+" + str(example_number.country_code) + random_phone_number)

In [ ]:
def randomDateBetween(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)
    
    return time.strftime(format, time.localtime(ptime))

In [ ]:
def create_row(column_definations):
    row_value = ""
    for x in column_definations:
        
        f_country_array = ["AU","CA","DE","ES","GB"] + ["CH"]*96
        
        f_country = random.choice(f_country_array)    
        
        #each if block if for datatype. You can add your own datatypes
        #e.g. datatype == "email" can be string.ascii_letters + string.digits + '@'
        if x["datatype"] == "string":
            if x["is_variable_legth"]:
                if x["is_null"]:
                    ll = random.randint(0,x["length"])
                else:
                    ll = random.randint(1,x["length"])
            else:
                ll = x["length"]
            value_of_string = ''.join(random.choice(string.ascii_letters + string.digits + 'äëöü') for _ in range(ll))
            value_of_string = '"' + value_of_string + '"'
        ####################
        elif x["datatype"] == "file":
            f= open(x["file_path"],encoding="utf8")
            test = []
            for row in f:
                test.append(row.rstrip("\n"))
            value_of_string=random.choice(test)
            value_of_string = '"' + value_of_string + '"'
        ####################
        
        elif x["datatype"] == "ssn":
            value_of_string = str(random.randrange(100,999)) + "-" + str(random.randrange(10,99)) + "-" + str(random.randrange(100,999))
            value_of_string = '"' + value_of_string + '"'
        ####################
        elif x["datatype"] == "number":
            value_of_string = str(random.randrange(10**int(x["min_range"]),10**int(x["max_range"])))
            value_of_string = '"' + value_of_string + '"'
        ####################
        elif x["datatype"] == "phonenumber":
            value_of_string = get_phonenumber(f_country)
            value_of_string = '"' + value_of_string + '"'
        ####################
        elif x["datatype"] == "xdate":
            value_of_string = randomDateBetween(x["from_date"], x["until_date"],x["date_format"], random.random())
            value_of_string = '"' + value_of_string + '"'
        ####################
        elif x["datatype"] == "country":
            #country_array = ["AD","AE","AF","AG","AI","AL","AM","AO","AQ","AR","AS","AT","AU","AW","AX","AZ","BA","BB","BD","BE","BF","BG","BH","BI","BJ","BL","BM","BN","BO","BQ","BR","BS","BT","BV","BW","BY","BZ","CA","CC","CD","CF","CG","CH","CI","CK","CL","CM","CN","CO","CR","CU","CV","CW","CX","CY","CZ","DE","DJ","DK","DM","DO","DZ","EC","EE","EG","EH","ER","ES","ET","FI","FJ","FK","FM","FO","FR","GA","GB","GD","GE","GF","GG","GH","GI","GL","GM","GN","GP","GQ","GR","GS","GT","GU","GW","GY","HK","HM","HN","HR","HT","HU","ID","IE","IL","IM","IN","IO","IQ","IR","IS","IT","JE","JM","JO","JP","KE","KG","KH","KI","KM","KN","KP","KR","KW","KY","KZ","LA","LB","LC","LI","LK","LR","LS","LT","LU","LV","LY","MA","MC","MD","ME","MF","MG","MH","MK","ML","MM","MN","MO","MP","MQ","MR","MS","MT","MU","MV","MW","MX","MY","MZ","NA","NC","NE","NF","NG","NI","NL","NO","NP","NR","NU","NZ","OM","PA","PE","PF","PG","PH","PK","PL","PM","PN","PR","PS","PT","PW","PY","QA","RE","RO","RS","RU","RW","SA","SB","SC","SD","SE","SG","SH","SI","SJ","SK","SL","SM","SN","SO","SR","SS","ST","SV","SX","SY","SZ","TC","TD","TF","TG","TH","TJ","TK","TL","TM","TN","TO","TR","TT","TV","TW","TZ","UA","UG","UM","US","UY","UZ","VA","VC","VE","VG","VI","VN","VU","WF","WS","YE","YT","ZA","ZM","ZW"]
            value_of_string = random.choice(f_country)
            value_of_string = '"' + value_of_string + '"'
        ####################
        elif x["datatype"] == "address":
            f_address= open(x["file_path"],encoding="utf8")
            address_array = []
            for row in f_address:
                address_array.append(row.rstrip("\n"))
            value_of_string=random.choice(address_array)
            value_of_string = '"' + value_of_string + '"'
        ####################        
        

        if row_value == "":
            row_value = value_of_string
        else:
            row_value = row_value + separator+ value_of_string
    return(row_value)

In [ ]:
%%timeit
f = open(data["filename"],'w',encoding="utf8")
for i in range(data["number_of_rows"]):
    each_row=create_row(data["columns"]).rstrip('\n')
    print(each_row ,file=f)
f.close()